<a href="https://colab.research.google.com/github/Clinda02/E-commerce_dataset_analysis/blob/main/Indian_Retail_dataset/Streamlit%20E-commerce%20Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Streamlit](https://streamlit.io/)을 이용한 대쉬보드 구축

# (1) 코드 함수 모듈화

In [8]:
# 패키지 설치
# !pip install streamlit
# !npm install localtunnel
# !pip install "ipywidgets>=7, <8"

In [15]:
# # 시각화 차트 컬러 문제
# import plotly.io as pio
# pio.templates.default = "plotly"

In [4]:
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np

In [1]:
from google.colab import drive
drive.mount('content/')

Mounted at content/


In [16]:
# 데이터 로드 함수 선언
def load_data():
  Olist = pd.read_csv('/content/data/List of Orders.csv')
  Detail = pd.read_csv('/content/data/Order Details.csv')
  data = Olist.merge(Detail, on = 'Order ID')

  return data

# 전처리 함수 선언
def preproc():
  data['Order Date'] = pd.to_datetime(data['Order Date'], format = '%d-%m-%Y')
  data['year'] = data['Order Date'].dt.year
  data['month'] = data['Order Date'].dt.month
  data['yearmonth'] = data['Order Date'].astype('str').str.slice(0,7)

  return data

In [17]:
data = load_data()
data = preproc()

# plotly 라인차트 함수 선언
def line_chart(data, x, y, title):
  df = data.groupby(x).agg({y : 'sum'}).reset_index()
  fig = px.line(df, x=x, y=y, title=title)
  fig.show()

  return fig

fig1 = line_chart(data, 'yearmonth', 'Quantity', 'Sales Quantity by month')
fig2 = line_chart(data, 'yearmonth', 'Amount', 'Sales Amount by month')

In [18]:
# plotly 바그래프 함수 선언
def bar_chart(data, x, y, title, color=None):
  if color is None :
    index = x
  else :
    index = [x, color]

  df = data.pivot_table(index=index, values=y, aggfunc='sum').reset_index()
  fig = px.bar(df, x=x, y=y, color = color, title=title)
  fig.show()

  return fig

fig3 = bar_chart(data, 'Category', 'Quantity', 'Sales Quantity by Category')
fig4 = bar_chart(data, 'yearmonth', 'Quantity', 'Monthly Sales Quantity by Category', 'Category')

In [19]:
# plotly 히트맵 함수 선언
def heatmap(data, x, y, z, title):
  df = data.pivot_table(index = ['State','Sub-Category'], values = ['Quantity', 'Amount', 'Profit'], aggfunc = 'sum').reset_index()
  fig = px.density_heatmap(df, x=x, y=y, z=z, title=title)
  fig.show()

  return fig

fig5 = heatmap(data, 'State', 'Sub-Category', 'Quantity', 'Quantity heat map')
fig6 = heatmap(data, 'State', 'Sub-Category', 'Amount', 'Amount heat map')

#(2) streamlit을 이용한 대시보드 구현

#### streamlit 함수
- `st.form`: 폼으로 영역 그룹화.  
- `st.columns`: 레이아웃 열 생성.  
- `st.expander`: 확장 가능한 토글.  
- `st.sidebar`: 사이드바에 요소 배치.  
- `st.tabs`: 탭으로 UI 구분.  
- `st.container`: 특정 블록 그룹화.  

In [40]:
%%writefile app.py

import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np

# 함수 모듈화
# 데이터 로드 함수 선언
@st.cache_data  # 데이터 및 시각화 출력값을 cache 메모리에 저장
def load_data():
  Olist = pd.read_csv('/content/data/List of Orders.csv')
  Detail = pd.read_csv('/content/data/Order Details.csv')
  data = Olist.merge(Detail, on = 'Order ID')

  return data

# 전처리 함수 선언
def preproc():
  data['Order Date'] = pd.to_datetime(data['Order Date'], format = '%d-%m-%Y')
  data['year'] = data['Order Date'].dt.year
  data['month'] = data['Order Date'].dt.month
  data['yearmonth'] = data['Order Date'].astype('str').str.slice(0,7)

  return data

# plotly line 차트 함수 선언
def line_chart(data, x, y, title):
  df = data.groupby(x).agg({y : 'sum'}).reset_index()
  fig = px.line(df, x=x, y=y, title=title)

  return df, fig

# plotly bar 그래프 함수 선언
def bar_chart(data, x, y, title, color=None):
  if color is None :
    index = x
  else :
    index = [x, color]

  df = data.pivot_table(index=index, values=y, aggfunc='sum').reset_index()
  fig = px.bar(df, x=x, y=y, color = color, title=title)

  return fig

# plotly 히트맵 함수 선언
def heatmap(data, x, y, z, title):
  df = data.pivot_table(index = ['State','Sub-Category'], values = ['Quantity', 'Amount', 'Profit'], aggfunc = 'sum').reset_index()
  fig = px.density_heatmap(df, x=x, y=y, z=z, title=title)

  return fig


# 메인 함수 선언
if __name__ == "__main__":
  # 대시보드 제목 설정
  st.title('E-Commerce Data 분석')
  st.write('시각화 대시보드 만들기')

  # 데이터 로드 및 전처리
  data = load_data()
  data = preproc()

  # 월별 판매량 분석
  st.subheader('월별 판매량 분석')
  with st.form('form', clear_on_submit = True): # clear_on_submit = True : 이전 결과 초기화(기본값 False)
      # 두 열의 버튼 생성
      col1, col2 = st.columns(2)
      submitted1 = col1.form_submit_button('판매량 그래프')
      submitted2 = col2.form_submit_button('매출액 그래프')

      if submitted1:  # 판매량 그래프
        df1, fig1 = line_chart(data, 'yearmonth', 'Quantity', 'Sales Quantity by month')
        st.dataframe(df1.T)
        st.plotly_chart(fig1, theme="streamlit", use_container_width=True) # use_container_width=True: 폼에 맞게 자동 크기 조절
      elif submitted2:  # 매출액 그래프
        df2, fig2 = line_chart(data, 'yearmonth', 'Amount', 'Sales Quantity by month')
        st.dataframe(df2.T)
        st.plotly_chart(fig2, theme="streamlit", use_container_width=True)

  # 품목별 판매량 분석
  st.subheader('품목별 판매량')
  col1, col2 = st.columns(2)
  with col1:  # 카테고리별 판매량
    col1.subheader("카테고리별 판매량")
    fig3 = bar_chart(data, 'Category', 'Quantity', 'Sales Quantity by Category')
    st.plotly_chart(fig3, theme="streamlit", use_container_width=True)
  with col2:  # 월별/카테고리별 누적 차트
    col2.subheader("월별/카테고리별 누적 차트")
    fig4 = bar_chart(data, 'yearmonth', 'Quantity', 'Monthly Sales Quantity by Category', 'Category')
    st.plotly_chart(fig4, theme="streamlit", use_container_width=True)

  # 지역별 판매 상품 분석
  st.subheader('지역별 주력 판매상품')
  tab1, tab2 = st.tabs(["Quantity heat map", "Amount heat map"])
  with tab1:  # 판매량 히트맵
    fig5 = heatmap(data, 'State', 'Sub-Category', 'Quantity', 'Quantity heat map')
    st.plotly_chart(fig5, theme="streamlit", use_container_width=True)
  with tab2:  # 매출액 히트맵
    fig6 = heatmap(data, 'State', 'Sub-Category', 'Amount', 'Amount heat map')
    st.plotly_chart(fig6, theme='streamlit', use_container_width=True)


Writing app.py


In [44]:
!streamlit run /content/app.py & npx lt --port 8501



⠙
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.145.171.1:8501

your url is: https://great-hoops-cheat.loca.lt
  Stopping...
^C


In [42]:
# @st.cache_data 캐시 초기화
!streamlit cache clear